<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_7_add_Traders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 5 and intro

Script Notes: 

(Have all cells run prior to recording and map loaded) 

In lesson six we finished agentizing the landscape, by creating our sugar and spice  agents, who were added to our schedule and landscape. Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again   

Now we want to add Trader agents who can harvest and trade the sugar and spice


Review each class and model set up

In [8]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import matplotlib.pyplot as plt



%matplotlib inline 

In [3]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (Epstein's rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): #part 3
        super().__init__(unique_id, model) #part 3
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

In [9]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn.
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice
       

**Part 2**:

As the Traders are more complicated agents we add parameters to initiate our agents we will add them here as well to ensure they both match.

First we need to do the standard unique_id and model with the super_init to inherit form the parent class. 

Then the random position we just initiated. As we are talking position, it is also immportnat to know the difference between [moore](https://en.wikipedia.org/wiki/Moore_neighborhood) and [von neuman](https://en.wikipedia.org/wiki/Von_Neumann_neighborhood) neighborhoods.  (show difference) 

Per Growing Artificial Societies we will use von neuman. As the keyword for mesa space is moore, we will mimic this and set moore to false

**Part 3**:

Now we add the endowments. now as we started key words must continue that or we will get an error, so first we add the sugar and then spice. For the agent this is how much sugr and spice they have so we will add to this as they trade and harvest and so just call them sugar and spice. 

**Part 4**:

Now we add metabolism using the key words metabolism sugar and metabolism spice. That we will pass in form the model

**Part 5**:

Finally we add vision

In [10]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos #part 2
      self.moore = False #part 2
      self.sugar = sugar #part 3
      self.spice = spice #part 3
      self.metabolism_sugar=metabolism_sugar, #part 4
      self.metabolism_spice=metabolism_spice, #part 4
      self.vision = vision #part 5

      

## 2 Add Traders

First to add traders we will add another key word argument, initial population. Again using Growing Artifical Societies as our model we will have an initial population of 100.  

---
**Part 1**: Now we create another for loop to instantiate our Trader agents. Make sure your for loop is at the same indentation level as the sugar and spice agents otherwise you could get 100 hundred Traders for every sugar and or spice agent created. 

---

**Part 2**:

For this loop we will use pythons range function, with the initial population attribute. Now we have to give the agents position, intial sugar and spice holdings and metabolism. For this we will use the same instantiations as growing artifical societies. As mesa `model.py` library uses pythons random library we will `self.random`, later we may add a seed to aid in results replication, but right now we not as the psuedo randomness can help with debugging. 

---
**Part 3:**

Now we add the intialize endowment for each agent. We will initialize these askeyword argument endowment_min and endowment max. This will also allow us to demonstrate batch_runner in a later lesson.  We then add this to our Trader object

**Part 4:**

Now each agents needs a metabolism and vision. Metabolisms determine how much sugar and spice each agent burns for each step. Again these are uniformly distributed so we will use the uniform distribution with key word argument. vision determines how many grids the agent can see in any given direction. So an agent with 5 can see 5 grid squares while an agent with one can anly see the grids squares next to the agent. As these have the same range of 1 -5 we will just call them capability min and max.

**Part 5:** 

Now we add the vision, a challenge here is random uniform returns a float while mesa get_neighbors which will take the vision paramter uses an integer, so we will have to convert the float into an integer

**Part 6**: Now that we have created each of the attributes we need to create the agents, add them to the scheduler and add them to the grid and then we make sure we iterate the agent

We will also add a print statement to see how our code is doing

In [15]:
class SugarscapeG1mt(mesa.Model):

  #part 0
  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min =1,
               metabolism_max = 5, capability_min=1, capability_max=5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population #part 0
    self.endowment_min = endowment_min #part3
    self.endowment_max = endowment_max #part3
    self.capability_min = capability_min #part4
    self.capability_max = capability_max #part4


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    sugar_distribution = np.genfromtxt("sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    # part 1
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) #part 2a
      y = self.random.randrange(self.height) #part 2a  
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) #part 3
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) #part 3
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.capability_min, self.capability_max)
      metabolism_spice = self.random.uniform(self.capability_min, self.capability_max)
      #add vision
      vision =int(self.random.uniform(self.capability_min, self.capability_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      print(trader.unique_id, trader.pos, trader.sugar, trader.metabolism_spice)
      agent_id += 1

        

In [ ]:
model = SugarscapeG1mt()

We have no complete initiating our model, next step is to create behvarios to watch the interactons proceed. 